In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/bangilhan3/train.csv')
train.head()
test = pd.read_csv('/kaggle/input/bangilhan3/public_test.csv')
test.head()

In [ ]:
# tokenizer 사용 - 띄어쓰기, 쉼표 등을 기준으로 텍스트 -> 숫자화
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()

# train과 test 모든 'text' 데이터 숫자화 
tk.fit_on_texts(list(train['text']) + list(test['text']))

# 숫자로 바뀐 단어 총 몇개 있는지 확인
tk.word_index # 숫자는 빈도에 따라 1,2,3,,,,
len(tk.word_index) # 숫자는 빈도에 따라 1,2,3,,,,

# 각 데이터 프레임 내 텍스트를 숫자화
train_text = tk.texts_to_sequences(train['text'])
test_text = tk.texts_to_sequences(test['text'])


# 가장 큰 숫자로 변한 텍스트는 길이 무엇인지
# ex) 가장 긴 길이를 가진 텍스트의 수로 맞춰준다
pd.Series(train_text).apply(lambda x : len(x)).max()
pd.Series(train_text).apply(len)


# 숫자로 변한 텍스트의 분포 확인
import matplotlib.pyplot as plt
import seaborn as sns

a, b = plt.subplots(1,1,figsize=(20,12)) # 밑그림
sns.distplot(pd.Series(train_text).apply(len)) # 대다수가 50개 정도 / train을 보면 test도 봐야함



In [ ]:
# 텍스트 길이가 제각각, 이에 어느정도 텍스트 길이를 맞춰주면 비교하는 데 더 효과적이지 않을까?

from keras.preprocessing.sequence import pad_sequences

# pad_test = pad_sequences(test_text, maxlen=256) 처음엔 256개의 단어가 있는 문장이 젤 길어서 이를 기준으로 다른 데이터도 256으로 맞춰줌
# pad_train = pad_sequences(train_text, maxlen=317)
# # pad는 알아서 의미없는 숫자를 넣어줌, 317로 바꿔주어야 해서 50에서 

# 근데 위 빈도 그래프에 따라서 100개에서 짤라도 크게 차이는 없을 것 같음 
pad_train = pad_sequences(train_text, maxlen=100, truncating ='post', padding= 'post') # truncating 기본 값은 pre
pad_test = pad_sequences(test_text, maxlen=100, truncating ='post', padding= 'post') # truncating 기본 값은 pre

In [ ]:
# 자 이제, train과 test 셋을 나눠보자! 여기서 포인트는 stratify인데, 스미싱이 전체 데이터 셋과 비교해 6%를 차지함. 이에, 트레인 테스트 셋 나눌 때도 6%를 유지

from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(pad_train, train['smishing'], test_size = 0.15, random_state=0, stratify=train['smishing']) #stratify 0,1 비율을 자동으로 맞춰줌

In [ ]:
# 딥러닝 모델을 쌓는 방법은 2가지
# 1. 순서대로 쌓는 , 
# 1) 불러오기
from keras import Sequential
from keras.layers import Dense, Embedding, Flatten # 가장 많이쓰는 층 #텍스트에서는 embedding #딥러닝에서는 구조 중요, 차원맞춰줌

# 2) 모델쌓기, 선언
model = Sequential()
model.add(Embedding(448903,25,input_length=100)) # 차원이 하나면 단어의 의미 분석이 제대로 안됨. 의미의 벡터를 늘려줄 필요가 있다. # 대회 시, 열심히 찾아나가야 되는 부분

#embedding(단어의 의미를 학습한다), 각 데이터에 접근해 스미싱인가 아닌가 *단어*를 보고 판단해야함 
model.add(Flatten()) # 2차원 -> 1차원 
model.add(Dense(2, activation='softmax')) # 옵션2개 중 1개는 정답 클래스 갯수
# 1일때는 activation='sigmoid'
model.compile(metrics = ['acc'], optimizer = 'adam', loss='sparse_categorical_crossentropy') # 3가지 옵션 넣어줄거임 * compile, 모델선언 완료
# 1일때는 binary_crossentropy
# sparse를 넣어주면 회귀로 인식하는 게 아닌 분류로 인식
# 최적화함수는 최적의 지점을 가는 여러 방법, adam을 주로씀
# metrics = ['acc'] 정확도 까지 볼 수 있음

from keras.callbacks import EarlyStopping, ModelCheckpoint # 라이브러리는 항상 대문자 / 로스가 안좋아지면 과적합으로 판단해 자동으로 멈춰주는 /
# modelcheckpoint - 모델에서 가장 잘 나온 가중치를 저장하는 
es = EarlyStopping(patience=2) # 이미지 같은 경우는 평가로스가 많이 튀는 경우가 있어서 5 이상 정도로 
mc = ModelCheckpoint('best.h5', save_best_only=True) # h5는 가중치를 저장하는 #최적의 순간만 저장하겠다

# 3) 모델 학습
# model.fit(x_train, y_train, epochs=100, validation_split=0.1,batch_size=128, callbacks=[es,mc]) #callback #batch size는 32가 기본 / 텍스트는 배치가 크면 결과 값이 안나옴, 이미지는 배치가 크면 좋음 
# x, y 설정 / 학습 횟수 결정, epoch / 학습(90%), 평가(10%)


model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid),batch_size=128, callbacks=[es,mc]) #callback #batch size는 32가 기본 / 텍스트는 배치가 크면 결과 값이 안나옴, 이미지는 배치가 크면 좋음 
# x, y 설정 / 학습 횟수 결정, epoch / 학습(90%), 평가(10%)




In [ ]:
# 최적 가중치 저장된 걸 가져와야함 
model.load_weights('best.h5')

In [ ]:
# predict
result = model.predict(pad_test, batch_size=128) #pad_test는 뒤쪽 짜른거 
result
# 왼쪽 / 오른쪽(), 스미싱 확률

In [ ]:
test['smishing'] = result[:,1]
sub = test[['id','smishing']]
sub.to_csv('baseline_평가셋 수정.csv', index=False)

In [ ]:
# 100개로 변경 / 차원 늘리기 / 최적 모델 저장한걸로 

In [ ]:
# 개선안 2가지 
# 1) 평가 셋을 뽑을 때 제대로(과학적) 뽑아야한다
# 2) 앞쪽 날아가는 부분 을 고려해 앞뒤 모두 봐주는 모델 필요

In [ ]:
# 앙상블
# 1) 이론 - 모델마다 학습방법(=동작하는 방법)이 다르다 (트리, 선형(거리기반), 인공신경망, 뉴럴 )
# 2) 왜 점수가 좋아지는 지 - 단점 보완이 가능(어떤 데이터셋의 70% 1번 모델, 80% 2번 모델, 1번 모델에서 틀린걸 2번에서는 맞춤 )
# 3) 정답값 10 10 10 
# 4) 모델1 예측 -> 13 9 11  모델1 에러 -> 5
# 5) 모델2 예측 -> 9 9 9  모델2 에러 -> 3
# 6) 모델1+모델2 예측 -> 11 9 10  모델1+모델2 에러 -> 2
# 조건 1) 2개의 모델의 점수가 비슷해야함
# 조건 2) 모델의 학습 방식이 최대한 달라야함. 비슷하면 안되고 88 89 -> 89.5 / 86 84 -> 90 후자의 케이스(학습 방식이 다른)가 점수가 높아짐 


# 딥러닝 모델을 어케 바꿀 수 없기 때문에, 데이터를 바꿔줌 => 앞쪽 데이터 날린거 + 뒤쪽 데이터 날린거 
# 앙상블 시 가중평균 형태로 더 나은 모델에 더 나은 가중을 실어줌


In [ ]:
first = pd.read_csv('/kaggle/output/working/baseline_평가셋 수정.csv')
second = pd.read_csv('/kaggle/input/taling/baseline_ .csv')
display(first.head())
display(second.head()) # 비교 시, 스미싱을 판단하는 수치 다름
# roc-auc평가방식 -> 확률이 중요한게 아닌, 상대적 순위가 중요해짐. 이에 두개의 앙상블 시, '순위'를 고려해야함 

In [ ]:
# 스미승으로 오름차순
first = first.sort_values('smishing')
second = second.sort_values('smishing')
display(first.head())
display(second.head()) # 0이 스미싱이 아니다 1이 스미싱

In [ ]:
 # 등수 부여 roc 평가방식은 상대적인 순위가 중요함
first['smishing'] = np.arange(1626) # 테스트셋의 숫자 0~ 1626 
second['smishing'] = np.arange(1626)
display(first.head())
display(second.head()) # 0이 스미싱이 아니다 1이 스미싱

In [ ]:
first = first.sort_values('id')
second = second.sort_values('id')
display(first.head())
display(second.head()) # 0이 스미싱이 아니다 1이 스미싱

# 1번 모델은 첫번째 데이터에 대해서 스미싱이 아니야 / 두번째는 스미싱 아니야
# 2번 모델은 첫번쨰 데이터에 대해 스미싱일 것 같아 / 두번째는 애매


In [ ]:
sub['smishing']= first['smishing']*0.75 + second['smishing']*0.25
sub.head()

In [ ]:
# smishing 최대값으로 나눠줌 
sub['smishing'] = sub['smishing']/1625
sub['smishing'].max()

In [ ]:
sub.to_csv('baseline_평가셋 앙상블2575 + 2.csv', index=False)

In [ ]:
점수를 '제출하기' 말고 엑셀 파일만 가지고 확인할 수 있는 방법은?


왜 테스트셋에서만 앙상블을 진행..?
트레인셋에서도 앙상블을 진행해
가중치를 더 낫게 만들 수는 없나?

앙상블은 단지 테스트셋 예측 값을 섞은것뿐인가? -> 앙상블 모델이 아닌 평가셋


왜 첫번째 모델에 0.75를 부여했찌